<a href="https://colab.research.google.com/github/Paco-Danes/CondGanMap2Sat/blob/main/TryOut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/TestSetEdi/test_tiles_edi.zip > /dev/null

In [ ]:
!unzip /content/drive/MyDrive/TestSetEdi/tiles_central-belt7500.zip> /dev/null

In [ ]:
EdiBigGen_PATH = ""
EdiRefDivergence_PATH = ""
CentralBeltBigGen_PATH = ""
SmallGenCentralBelt_PATH = ""

In [ ]:
#@title Load models and test set for evaluation.
# Load models trained on Edinburgh dataset
BigGenEdi = BigGen(num_classes=3).cuda()
DivGenEdi = BigGen(num_classes=3).cuda()
cp_BigGenEdi = torch.load(EdiBigGen_PATH)
cp_DivGenEdi = torch.load(EdiRefDivergence_PATH)
BigGenEdi.load_state_dict(cp_BigGenEdi['gen_state_dict'])
DivGenEdi.load_state_dict(cp_DivGenEdi['gen_state_dict'])

# Load models trained on CentralBelt (50'000) dataset
BigGenCent = BigGen(num_classes=3).cuda()
cp_BigGenCent = torch.load(CentralBeltBigGen_PATH)
BigGenCent.load_state_dict(cp_BigGenCent['gen_state_dict'])
SmallGenCent = Generator(num_classes=3).cuda()
cp_SmallGenCent = torch.load(SmallGenCentralBelt_PATH)
SmallGenCent.load_state_dict(cp_SmallGenCent['gen_state_dict'])

# Set the models to evaluation mode
BigGenEdi.eval()
DivGenEdi.eval()
BigGenCent.eval()

# Double gan: BigGen + refinement
divergenceNet = nn.Sequential(
    BigGenEdi,
    DivGenEdi
)

# Move the sequential model to GPU if available
divergenceNet = divergenceNet.cuda()

transform = transforms.Compose([
    transforms.ToTensor(),
])

# Replace 'source_folder' and 'target_folder' with your actual folder paths
test_dataset_edi = InMemoryCustomDataset(source_dir='/content/tiles_edi1000/openstreetmap',
                                      target_dir='/content/tiles_edi1000/worldimagery-clarity',
                                      transform=transform,
                                      augment_rotation=False)

test_dataset_central = InMemoryCustomDataset(source_dir='/content/tiles_central-belt7500/openstreetmap',
                                      target_dir='/content/tiles_central-belt7500/worldimagery-clarity',
                                      transform=transform,
                                      augment_rotation=False)


print("Number of test images-edi: ", len(test_dataset_edi))
print("Number of test images-central: ", len(test_dataset_central))

In [ ]:
# @title Test images Show-Off, comparison between 2 models
def showOff(first_m, second_m, data):
  # Get a batch of images from the test loader
  test_loader = DataLoader(data, batch_size=6, shuffle=True)
  data_iterator = iter(test_loader)
  images = next(data_iterator)

  # Extract source images
  source_images = images['source_image'].to('cuda')
  # Generate target imgs using source imgs with both models, BigGen and BigGen+Refinement
  with torch.no_grad():
      gen_no_ref = first_m(source_images)
      generated_targets = second_m(source_images)

  # Plot everything, rows: source; BigGen; BigGen + Refinement
  fig, axes = plt.subplots(3, 6, figsize=(12, 7))

  for i in range(6):
      source_img = source_images[i].cpu().permute(1, 2, 0)
      gen_img = gen_no_ref[i].cpu().permute(1, 2, 0)
      gen_img_ref = generated_targets[i].cpu().permute(1, 2, 0)

      axes[0, i].imshow(source_img)
      axes[0, i].axis('off')
      axes[1, i].imshow(np.clip(gen_img, 0, 1))
      axes[1, i].axis('off')
      axes[2, i].imshow(np.clip(gen_img_ref, 0, 1))
      axes[2, i].axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
# Edinburgh: SingleGan Vs Gan+Refinement
showOff(BigGenEdi, divergenceNet, test_dataset_edi)

In [ ]:
# Central Belt, Small Generator vs Big Generator
showOff(SmallGenCent, BigGenCent,  test_dataset_central)